In [1]:
from zpmeta.superclasses.panelcachedsource import PanelCachedSource
from zpmeta.superclasses.functionclass import FunctionClass
from zpmeta.metaclasses.singletons import MultitonMeta
from pandas import DataFrame, Series, concat, MultiIndex, date_range, IndexSlice
import numpy as np
from datetime import datetime
import logging

logging.basicConfig(level=logging.INFO)

In [2]:
class RandomPanelCachedSource(PanelCachedSource, metaclass=MultitonMeta):
    '''Subclasses PanelCachedSource to create a dataframe of random numbers.
    Accepts a dictionary of parameters, including:
    cols: list of column names
    '''
    def __init__(self, params: dict = None):
        super(RandomPanelCachedSource, self).__init__(params)
        self.appendable = dict(xs=True, ts=True)
    
    def execute(self, call_type=None, entities=None, period=None):
        cols = MultiIndex.from_product([val for val in entities.values()], names=entities.keys())
        idx = date_range(period[0], period[1], freq=self.params['freq'])
        result = DataFrame(np.random.randn(len(idx), len(cols)), columns=cols, index=idx)
        
        return result
    

In [3]:
class Multiply_g_DataFrame(FunctionClass):
    @classmethod
    def _std_params(cls, name: str = None) -> dict:
        return dict(f = 1.0)

    def execute(self, operand: DataFrame, period=None, params=None) -> DataFrame:
        return operand * params['f']

In [4]:
daily_df_source = RandomPanelCachedSource(dict(freq='B'))
df = daily_df_source(entities=dict(Type=['A','B','C'], ID=[1,2]), period=(datetime(2019,1,12), datetime(2019,1,31)))
df 

INFO:root:args: ({'freq': 'B'},) ; kwds: {}
INFO:root:Multiton checking registry for key: (<class '__main__.RandomPanelCachedSource'>, '{"freq": "B"}')
INFO:root:Multiton No Instance of <class '__main__.RandomPanelCachedSource'> {"freq": "B"}
INFO:root:Multiton Registering Instance of <class '__main__.RandomPanelCachedSource'> {"freq": "B"}
INFO:root:RUN RandomPanelCachedSource {'freq': 'B'}
INFO:root:RUN INITIAL: [{'Type': ['A', 'B', 'C'], 'ID': [1, 2]}] 2019-01-12 00:00:00 - 2019-01-31 00:00:00
INFO:root:EXEC INITIAL: [{'Type': ['A', 'B', 'C'], 'ID': [1, 2]}] 2019-01-12 00:00:00 - 2019-01-31 00:00:00
INFO:root:DONE RandomPanelCachedSource {'freq': 'B'}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14  0.349255  0.918312  0.026817  0.947622  0.913788 -0.366277
2019-01-15  0.274065  1.769669 -0.023055  1.999293  0.372711 -0.312733
2019-01-16  1.330708 -1.226153 -1.568479 -0.272704 -1.244243 -0.001249
2019-01-17  0.859802  0.944945  0.409960 -1.034144 -0.155895  1.048444
2019-01-18  0.052254  0.320357  1.835271 -0.174857  0.949358 -1.619255
2019-01-21  0.870655  0.291410  0.765339  0.740674 -0.820050 -0.500600
2019-01-22 -0.348856 -0.653752 -0.303507 -0.918037  0.546856 -0.454785
2019-01-23 -0.855336  1.329794 -0.450928 -0.819109  1.373180  0.634611
2019-01-24 -2.203532  0.604911  0.409724  1.725433  1.418685  1.597013
2019-01-25  0.441282 -0.923701 -0.893023 -0.238018 -1.113700  1.869680
2019-01-28  0.800787  0.549549 -0.231230 -0.774071  0.908102  0.001961
2019-01-29  0.645744  0.627320  0.981150 -0.273610  1.960056  0.474732
2019-01-30 -0.030812 -0.710267 -1.236601 -0.926424 -0.529085  0.289299
2019-01-31 -0.356687  0.710619 -1.094396 -1.576837  0.260859  0.232375

In [5]:
multiply = Multiply_g_DataFrame(dict(f=2))
df2 = multiply(df)
df2

INFO:root:INIT Multiply_g_DataFrame {'f': 2}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14  0.698511  1.836624  0.053634  1.895245  1.827577 -0.732555
2019-01-15  0.548129  3.539338 -0.046110  3.998587  0.745421 -0.625466
2019-01-16  2.661416 -2.452307 -3.136959 -0.545408 -2.488486 -0.002497
2019-01-17  1.719604  1.889890  0.819919 -2.068288 -0.311789  2.096888
2019-01-18  0.104508  0.640713  3.670542 -0.349714  1.898716 -3.238510
2019-01-21  1.741309  0.582819  1.530678  1.481348 -1.640101 -1.001199
2019-01-22 -0.697712 -1.307503 -0.607014 -1.836074  1.093713 -0.909571
2019-01-23 -1.710673  2.659587 -0.901857 -1.638218  2.746360  1.269223
2019-01-24 -4.407063  1.209821  0.819447  3.450865  2.837370  3.194026
2019-01-25  0.882564 -1.847403 -1.786046 -0.476036 -2.227401  3.739360
2019-01-28  1.601575  1.099098 -0.462460 -1.548141  1.816205  0.003923
2019-01-29  1.291488  1.254640  1.962300 -0.547220  3.920111  0.949464
2019-01-30 -0.061623 -1.420534 -2.473202 -1.852848 -1.058169  0.578598
2019-01-31 -0.713375  1.421239 -2.188793 -3.153673  0.521717  0.464750

We can override the default params while calling the FunctionClass instance

In [9]:
df3 = multiply(df, params={'factor': 3})
df3

Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14  0.698511  1.836624  0.053634  1.895245  1.827577 -0.732555
2019-01-15  0.548129  3.539338 -0.046110  3.998587  0.745421 -0.625466
2019-01-16  2.661416 -2.452307 -3.136959 -0.545408 -2.488486 -0.002497
2019-01-17  1.719604  1.889890  0.819919 -2.068288 -0.311789  2.096888
2019-01-18  0.104508  0.640713  3.670542 -0.349714  1.898716 -3.238510
2019-01-21  1.741309  0.582819  1.530678  1.481348 -1.640101 -1.001199
2019-01-22 -0.697712 -1.307503 -0.607014 -1.836074  1.093713 -0.909571
2019-01-23 -1.710673  2.659587 -0.901857 -1.638218  2.746360  1.269223
2019-01-24 -4.407063  1.209821  0.819447  3.450865  2.837370  3.194026
2019-01-25  0.882564 -1.847403 -1.786046 -0.476036 -2.227401  3.739360
2019-01-28  1.601575  1.099098 -0.462460 -1.548141  1.816205  0.003923
2019-01-29  1.291488  1.254640  1.962300 -0.547220  3.920111  0.949464
2019-01-30 -0.061623 -1.420534 -2.473202 -1.852848 -1.058169  0.578598
2019-01-31 -0.713375  1.421239 -2.188793 -3.153673  0.521717  0.464750

Now let us use the pre feature. Let us first define a FunctionClass that calculates exponents

In [6]:
class Exponent_g_DataFrame(FunctionClass):
    @classmethod
    def _std_params(cls, name: str = None) -> dict:
        return dict(f = 1.0)

    def execute(self, operand: DataFrame, period=None, params=None) -> DataFrame:
        return operand**params['f']

Now let us chain these two function classes to create a function that first squares a DataFrame and then multipes by 2.

In [8]:
double_squared = Multiply_g_DataFrame(dict(f=2), pre=Exponent_g_DataFrame(dict(f=2)))
df3 = double_squared(df)
df3

INFO:root:INIT Exponent_g_DataFrame {'f': 2}
INFO:root:INIT Multiply_g_DataFrame {'f': 2}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14  0.243959  1.686593  0.001438  1.795976  1.670018  0.268318
2019-01-15  0.150223  6.263456  0.001063  7.994348  0.277827  0.195604
2019-01-16  3.541567  3.006905  4.920256  0.148735  3.096282  0.000003
2019-01-17  1.478519  1.785842  0.336134  2.138907  0.048606  2.198471
2019-01-18  0.005461  0.205257  6.736439  0.061150  1.802561  5.243972
2019-01-21  1.516079  0.169839  1.171487  1.097196  1.344966  0.501200
2019-01-22  0.243401  0.854783  0.184233  1.685584  0.598104  0.413659
2019-01-23  1.463201  3.536702  0.406673  1.341879  3.771247  0.805463
2019-01-24  9.711104  0.731834  0.335747  5.954236  4.025334  5.100900
2019-01-25  0.389459  1.706448  1.594979  0.113305  2.480657  6.991406
2019-01-28  1.282521  0.604008  0.106935  1.198371  1.649300  0.000008
2019-01-29  0.833971  0.787060  1.925310  0.149725  7.683636  0.450741
2019-01-30  0.001899  1.008958  3.058363  1.716522  0.559861  0.167388
2019-01-31  0.254452  1.009960  2.395407  4.972827  0.136095  0.107996